In [ ]:
!pip install pandas
!pip install matplotlib

### prices

In [ ]:
import pandas as pd

# src: https://www.investing.com/crypto/bitcoin/btc-usd-historical-data
def get_prices():
    # Чтение CSV файла
    df = pd.read_csv('BTC_USD_Bitfinex_2012-2024.csv', delimiter=',', thousands=',', decimal='.')
    return df

prices = get_prices()
prices = prices[::-1].reset_index(drop=True)
prices['Date'] = pd.to_datetime(prices['Date'])

### halvings

In [ ]:
import numpy as np
from datetime import datetime, timedelta

# Constants for the Bitcoin reward halving
initial_reward = 50  # initial block reward
halving_interval = 210000  # blocks per halving

# Calculate the halving dates and rewards
halving_dates = [
    datetime(2009, 1, 3),  # Genesis block
    datetime(2012, 11, 28),
    datetime(2016, 7, 9),
    datetime(2020, 5, 11),
    datetime(2024, 4, 19)
]

halving_dates

In [ ]:
# Calculate reward per block after each halving
rewards = [initial_reward / (2 ** i) for i in range(len(halving_dates))]
rewards

### stock to flow

In [ ]:
def get_stock_to_flow():
    # Constants for the Bitcoin reward halving
    halving_interval = 210000  # blocks per halving
    # Create an empty list to hold the data
    data = []
    total_mined = 0  # Initial total mined
    # Generate the dataset
    for i in range(len(halving_dates)):
        if (i < len(halving_dates) - 1):
            # print(f'debug: True, {i}, {len(halving_dates)}')
            start_date = halving_dates[i]
            end_date = halving_dates[i + 1]
            date_range = pd.date_range(start=start_date, end=end_date - timedelta(days=1), freq='D')
            # Calculate blocks per day for this period
            days_in_period = (end_date - start_date).days
            blocks_per_day = halving_interval / days_in_period
            current_reward = rewards[i]
            for single_date in date_range:
                daily_mined = blocks_per_day * current_reward
                total_mined += daily_mined
                yearly_mined = 365 * daily_mined
                stock_to_flow = total_mined / yearly_mined
                data.append([single_date, total_mined, daily_mined, yearly_mined, current_reward, stock_to_flow])
        else:
            # print(f'debug: False, {i}, {len(halving_dates)}')
            start_date = halving_dates[i]
            end_date = prices.iloc[-1]['Date']
            date_range = pd.date_range(start=start_date, end=end_date - timedelta(days=1), freq='D')
            # fixed 144 blocks per day for the last (incomplete) period.
            blocks_per_day = 144
            current_reward = rewards[i]
            for single_date in date_range:
                daily_mined = blocks_per_day * current_reward
                total_mined += daily_mined
                yearly_mined = 365 * daily_mined
                stock_to_flow = total_mined / yearly_mined
                data.append([single_date, total_mined, daily_mined, yearly_mined, current_reward, stock_to_flow])

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=['Date', 'Total Mined', 'Daily Mined', 'Yearly Mined', 'Current Reward', 'Stock to Flow'])
    return df

# Generate and display the dataframe
df = get_stock_to_flow()
df


### charts

In [ ]:
len(df)

In [ ]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(figsize=(10, 6))

first_price_date = prices['Date'].iloc[0]
s2f = df[df['Date'] >= first_price_date]

# Первая ось для Stock to Flow
ax1.plot(s2f['Date'], np.log10(s2f['Stock to Flow']), label='Stock to Flow', color='b')
ax1.set_xlabel('Date')
ax1.set_ylabel('Stock to Flow', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Вторая ось для цены
ax2 = ax1.twinx()
ax2.plot(prices['Date'], np.log10(prices['Price']), label='Price', color='g')
ax2.set_ylabel('Price', color='g')
ax2.tick_params(axis='y', labelcolor='g')

# Добавляем заголовок и легенду
plt.title('Stock to Flow and Price over Time')
fig.tight_layout()  # Для предотвращения наложения осей
plt.show()


In [ ]:
# now lets try to reproduce the result in the PlanB papper where there is a liniar depencency 
# between log price and log stock-to-flow ratio.

# Убедимся, что есть только те даты, которые присутствуют в обоих DataFrame
merged_df = pd.merge(prices[['Date', 'Price']], df[['Date', 'Stock to Flow']], on='Date', how='inner')

# Строим график
plt.figure(figsize=(6, 6))
plt.loglog(merged_df['Stock to Flow'], merged_df['Price'], label='Price vs. Stock to Flow')

# Добавляем подписи к осям и заголовок
plt.xlabel('Stock to Flow (log scale)')
plt.ylabel('Price (log scale)')
plt.title('Price vs. Stock to Flow with Logarithmic Scales')
plt.legend()
plt.grid(True, which="both", ls="--")  # Логарифмическая сетка
plt.show()
